Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

Maastricht_University_logo.svg


# Information Retrieval and Text Mining Course
## Tutorial 12 — Conversational Search: Agentic Approaches

**Author:** Jan Scholtes

**Edition 2025-2026**

Department of Advanced Computer Sciences — Maastricht University

Welcome to Tutorial 12 on **Agentic Approaches for Conversational Search**. AI has been interested in Agents for quite some time. Already in 1995's *Artificial Intelligence: A Modern Approach* ([AIMA](https://en.wikipedia.org/wiki/Artificial_Intelligence:_A_Modern_Approach)) by Russell and Norvig, an 'agent' is defined as anything that (i) senses its environment, (ii) perceives inputs, (iii) responds through reasoning, and (iv) acts upon the environment. Today, LLMs make many of these original agent goals practical.

In this tutorial we explore how **LLM-based agents** extend standard Retrieval-Augmented Generation (RAG) to create truly autonomous conversational search systems. The topics covered are:

1. **Why Agentic Architecture?** — limitations of standard RAG vs. agent-based search.
2. **The 7 Components of an LLM-Based Agent** — perception, memory, action interface, goal management, reasoning & planning, learning, verification & guardrails.
3. **Perception** — LLMs as multimodal input processors.
4. **Memory** — short-term, long-term (FAISS), and episodic memory.
5. **Action Interface & Tools** — the ReAct paradigm (Reason + Act).
6. **Goal & Task Management, Reasoning & Planning** — planner + worker agents.
7. **Verification & Guardrails** — critic agents, fact-checking, safety filters.
8. **Multi-Agent Orchestration** — manager/specialists, debate/committee, shared blackboard.
9. **Agent Interoperability & Standards** — MCP, A2A, LangGraph, OpenAI Agents SDK.
10. **Challenges & Future Directions** — safety, evaluation, multimodal search.

At the end you will find the **Exercises** section with graded assignments.

> **Note:** This course is about Information Retrieval, Text Mining, and Conversational Search — not about programming skills. The code cells below show you *how* agentic architectures work in practice using the **OpenAI Agents SDK**. Focus on understanding the **concepts** and **results**.

> **Important:** This tutorial requires an **OpenAI API key** with billing enabled. The cost is typically €1–€3 for the entire tutorial when using `gpt-4o-mini`.

## Library Installation

We install all required packages in a single cell. Run this cell once at the beginning of your session.

In [ ]:
# Install required packages
import subprocess, sys

packages = [
    "openai>=1.40.0",
    "openai-agents",
    "faiss-cpu",
    "tiktoken",
    "numpy",
]
for pkg in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print("All packages installed successfully.")

## Library Imports

All imports are grouped here so the notebook is easy to set up and run.

In [ ]:
# Core Python
import os
import json
import time
import asyncio
import getpass
from datetime import datetime, timezone

# Numerical
import numpy as np

# OpenAI client
from openai import OpenAI

# OpenAI Agents SDK
from agents import Agent, Runner, function_tool

# FAISS for vector memory
import faiss

print("All libraries imported successfully.")

## Setting Up the OpenAI API Key

To run this tutorial, you need an **OpenAI API key** with an activated billing method. Follow these steps:

1. **Create an OpenAI account** at [https://platform.openai.com/](https://platform.openai.com/)
2. **Create a new API key** at [https://platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)
3. **Add credits / enable billing** at [https://platform.openai.com/settings/organization/billing/overview](https://platform.openai.com/settings/organization/billing/overview) (typically €1–€3 for this tutorial with `gpt-4o-mini`)

⚠️ **Never share your API key or commit it to version control.** The cell below will prompt you securely.

In [ ]:
# API Key Setup — enter your key when prompted (input is hidden)
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

assert "OPENAI_API_KEY" in os.environ and os.environ["OPENAI_API_KEY"], \
    "Please set OPENAI_API_KEY as an environment variable."

client = OpenAI()

# Quick verification — list a few available models
models = client.models.list()
print("API key verified! Available models (first 5):")
print([m.id for m in models.data][:5])

---
# 1. Why Agentic Architecture for Search?

## Standard RAG Is Not Enough

Standard RAG follows a rigid, linear pipeline: **Query → Retrieve → Generate**. While effective for simple factual questions, it has fundamental limitations:

| Limitation | Description |
|---|---|
| **Static Retrieval** | No self-correction if the wrong documents are fetched. The system cannot decide to search again with different terms. |
| **Context Fragmentation** | Struggles with long-term reasoning across many conversation turns or across multiple sources. |
| **Lack of Proactivity** | Cannot independently call tools, verify facts, or ask clarifying questions. |
| **One-Shot Reasoning** | Answers in a single pass — no iterative refinement, no multi-hop reasoning. |

## What Agentic Architecture Achieves

| Dimension | RAG + LLM | Agentic Search |
|---|---|---|
| **Workflow** | Fixed pipeline | Dynamic, adaptive loops |
| **Decision Making** | None — follows fixed steps | Autonomous — decides what to do next |
| **Data Sources** | Pre-configured retriever | Can discover and query multiple sources |
| **Verification** | None built-in | Self-checking, critic agents, guardrails |

## Applications Only Possible with Agents

1. **Autonomous Research & Synthesis** — multi-hop queries across domains (e.g., "Map the landscape of autonomous agents in legal tech between 2023–2025")
2. **End-to-End Troubleshooting** — diagnose problems, call APIs, execute fixes
3. **Strategic Decision Support** — break down complex business questions into sub-analyses
4. **Forensics & Compliance Monitoring** — real-time pattern detection across large text corpora

---
# 2. From LLM to Agent — The 7 Components

LLMs become agents when placed in a loop with memory, goals, reasoning, actions, and feedback. To go from a stand-alone LLM to an agent, we wrap the LLM with seven components:

| # | Component | Description |
|---|---|---|
| 1 | **Perception** | Text, speech, images, structured data — multimodal input processing |
| 2 | **Memory** | Short-term (context window), long-term (vector DBs like FAISS/Pinecone), episodic (interaction logs) |
| 3 | **Action Interface / Tools** | API calls, code execution, database queries — follows the **ReAct** paradigm (Reason + Act) |
| 4 | **Goal / Task Management** | Natural-language goals, decomposition into subgoals, dynamic reprioritization |
| 5 | **Reasoning & Planning** | CoT (Chain-of-Thought), ToT (Tree-of-Thought), SoT (Skeleton-of-Thought), explicit planners, control loops |
| 6 | **Learning & Adaptation** | Fine-tuning, RLHF (Reinforcement Learning from Human Feedback), process-based rewards |
| 7 | **Verification & Guardrails** | Fact-checking modules, safety filters, external critics, debate agents |

### Why LLMs as Agents?

- **Natural language interface** — text is expressive enough to encode plans, decisions, and feedback
- **General world knowledge** — no handcrafted rules needed
- **Few-shot adaptability** — can adapt behaviour via prompting
- **Scalability** — once an agent loop is defined, it works across domains

### The Symbiosis

LLMs make many 1990s agent dreams practical. Conversely, agentic architectures fix LLM shortcomings (hallucinations, no tool access, limited context, one-shot reasoning, no self-critique).

We will now demonstrate each component with working code using the **OpenAI Agents SDK**.

---
# 3. Perception

LLMs naturally handle diverse inputs expressed as text: instructions, structured data, dialogue. Multimodal models extend this to images, audio, and video. With today's LLMs, input processing functionality is built-in: chatting, speech, images, document uploads (including tables in XLS and OCR for images and bitmap PDFs).

Until the introduction of ChatGPT (November 30, 2022), this was absolutely NOT trivial. Most NLP models before that time failed to deliver this functionality reliably.

Let's demonstrate the LLM's perception capabilities by giving it mixed natural language + structured data:

In [ ]:
# Perception: LLM as multimodal text processor
prompt = """You will receive:
- Natural language instruction
- A tiny piece of structured data

Explain what the user wants, then compute the answer.

User instruction: "Compute the mean and max of these scores."
Scores (JSON): [7.5, 8.0, 9.0]
"""

response = client.responses.create(
    model="gpt-4o-mini",
    input=prompt,
)
print(response.output_text)

**Observation:** The LLM seamlessly processes the mixed natural-language instruction and structured JSON data. It understands *what* the user wants (compute statistics) and *how* to do it (arithmetic). This demonstrates the **perception** component — the agent's ability to understand diverse input formats without special parsers.

---
# 4. Memory

Stand-alone LLMs are **stateless**. Once the prompt ends, they forget everything. For an agent to function, we need external memory of several kinds:

| Type | Description | Implementation |
|---|---|---|
| **Short-term** | Current conversation history | Context window of the LLM |
| **Long-term** | Persistent, retrievable knowledge | External vector databases (FAISS, Pinecone, Chroma) |
| **Episodic** | Logs of prior interactions with timestamps | Structured logs retrievable by time/context |
| **Semantic** | Retrieved knowledge chunks | RAG (Retrieval-Augmented Generation) |

Let's demonstrate each type.

## 4.1 Short-Term Memory

Short-term memory is maintained through the conversation context. The OpenAI Agents SDK provides `SQLiteSession` to persist session state locally. The agent remembers what was said earlier in the conversation:

In [ ]:
from agents import SQLiteSession

chat_agent = Agent(
    name="ShortTermAssistant",
    instructions="You are a concise assistant. Remember details from this conversation.",
    model="gpt-4o-mini",
)

session = SQLiteSession("irtm-demo-session")

# Turn 1: introduce ourselves
result1 = await Runner.run(
    chat_agent,
    "My name is Alex and I work on legal NLP.",
    session=session,
)
print("Turn 1:", result1.final_output)

# Turn 2: test whether the agent remembers
result2 = await Runner.run(
    chat_agent,
    "What is my research area, and what is my name?",
    session=session,
)
print("\nTurn 2:", result2.final_output)

**Observation:** The agent remembers "Alex" and "legal NLP" from Turn 1 when asked about it in Turn 2. This short-term memory is maintained by the `SQLiteSession`, which automatically manages the conversation history within the context window.

## 4.2 Long-Term Memory (FAISS Vector Store)

An example of long-term memory is using **FAISS** to store information as vector embeddings in an external index. This goes beyond the limited context window and allows the agent to recall facts from arbitrarily many past interactions:

In [ ]:
# Long-term memory with FAISS
embedding_model = "text-embedding-3-small"

def embed(texts):
    """Embed a list of texts using OpenAI's embedding API."""
    res = client.embeddings.create(model=embedding_model, input=texts)
    return np.array([d.embedding for d in res.data], dtype="float32")

# Create FAISS index (inner-product similarity)
dim = len(embed(["test"])[0])
index = faiss.IndexFlatIP(dim)
memory_texts: list[str] = []

def add_memory(text: str):
    """Store a fact in long-term vector memory."""
    vec = embed([text])
    index.add(vec)
    memory_texts.append(text)

def search_memory(query: str, k: int = 3):
    """Retrieve the k most relevant facts from long-term memory."""
    if index.ntotal == 0:
        return []
    qvec = embed([query])
    scores, ids = index.search(qvec, min(k, index.ntotal))
    return [(memory_texts[i], float(scores[0][j])) for j, i in enumerate(ids[0]) if i >= 0]

# Populate memory with some facts
add_memory("Jan Scholtes teaches Information Retrieval and Text Mining at Maastricht University.")
add_memory("The course covers retrieval models, text classification, topic modeling, and agentic search.")
add_memory("FAISS is a library from Meta for efficient similarity search in dense vector collections.")

# Search for a relevant fact
results = search_memory("What course is taught at Maastricht University?")
print("Long-term memory search results:")
for text, score in results:
    print(f"  ({score:.4f}) {text}")

## 4.3 Episodic Memory

In human cognition, **episodic memory** (Tulving, 1983) stores personal experiences tied to time and context. LLMs have **no built-in episodic memory** — the model weights do not update after each conversation.

However, we can *approximate* episodic memory by logging interactions with timestamps and metadata. This gives us a time-ordered personal history that can be searched later:

In [ ]:
# Approximate episodic memory with timestamped logs
conversation_log = []

def log_event(event: str):
    """Log an event with a UTC timestamp."""
    timestamp = datetime.now(timezone.utc).isoformat()
    conversation_log.append(f"{timestamp} — {event}")

# Log some events
log_event("Started tutorial notebook")
log_event("Added three long-term memory entries about the IRTM course")
log_event("Searched long-term memory for course information")

print("Episodic memory log:")
for entry in conversation_log:
    print(f"  {entry}")

**Key Takeaway on Memory:**

| Memory Type | LLM Capability | Agent Enhancement |
|---|---|---|
| Short-term | Context window (limited tokens) | Session persistence (SQLiteSession) |
| Long-term | None — stateless | Vector databases (FAISS, Pinecone) |
| Episodic | None — no autobiographical timeline | Timestamped interaction logs |
| Semantic | Training data (static, cut-off) | RAG with live retrieval |

Without agentic memory wrappers, LLMs forget everything between sessions.

---
# 5. Action Interface & Tools

The key difference between an LLM and an Agent is that **an Agent actually does things** — it doesn't just talk. We need:

- **Tool calling** — APIs, Python execution, database queries
- **Plugins / function calling** — as in OpenAI's tool APIs
- **Environment interaction** — robotics controllers, operating systems

With tools, the agent can reach **beyond its own limitations**. This follows the **ReAct paradigm** (Yao et al., 2022): the agent alternates between *Reasoning* (thinking about what to do) and *Acting* (calling tools):

**Thought** → **Action** (tool call) → **Observation** → repeat until **Answer**

Let's define some tools and create a ReAct-style agent:

In [ ]:
# Define tools that the agent can call

@function_tool
def safe_calculator(expression: str) -> str:
    """
    Evaluate a simple arithmetic expression using +, -, *, / and parentheses.
    Used as a tool by the agent instead of letting the LLM 'hallucinate' arithmetic.
    """
    import math
    allowed = set("0123456789+-*/(). ")
    if any(ch not in allowed for ch in expression):
        return "Error: disallowed characters."
    try:
        value = eval(expression, {"__builtins__": {}}, {"math": math})
        return str(value)
    except Exception as e:
        return f"Error while evaluating: {e}"

@function_tool
def store_memory(text: str) -> str:
    """Store a fact in the shared long-term vector memory."""
    add_memory(text)
    log_event(f"Stored in memory: {text}")
    return f"Stored: {text}"

@function_tool
def retrieve_memory(query: str) -> str:
    """Retrieve relevant facts from the shared long-term vector memory."""
    results = search_memory(query, k=3)
    if not results:
        return "No relevant memories found."
    return "\n".join(f"- {text} (score={score:.3f})" for text, score in results)

print("Tools defined: safe_calculator, store_memory, retrieve_memory")

In [ ]:
# ReAct agent: Reason + Act in a loop
tool_agent = Agent(
    name="ReActDemoAgent",
    model="gpt-4o-mini",
    instructions="""You are an LLM-based agent for the Information Retrieval and Text Mining course 
at Maastricht University. You can:
- Store important facts in long-term memory (store_memory).
- Retrieve memories (retrieve_memory).
- Use a calculator (safe_calculator).
Think step-by-step (Reason), then decide whether to call a tool (Act).
Explain briefly what you're doing in natural language.""",
    tools=[safe_calculator, store_memory, retrieve_memory],
)

session_tools = SQLiteSession("tools-demo")

# Query 1: Store a fact
q1 = "Please remember that my research topic is 'Information extraction for legal case law.'"
r1 = await Runner.run(tool_agent, q1, session=session_tools)
print(f"Q1: {q1}")
print(f"A1: {r1.final_output}")

print("\n" + "="*60 + "\n")

# Query 2: Retrieve memory + compute
q2 = "What is my research topic? Also compute (17 * 23 - 5) / 4."
r2 = await Runner.run(tool_agent, q2, session=session_tools)
print(f"Q2: {q2}")
print(f"A2: {r2.final_output}")

**Observation:** The agent demonstrates the **ReAct pattern** — it reasons about what it needs to do, then calls the appropriate tools. For the memory question, it uses `retrieve_memory`. For arithmetic, it uses `safe_calculator` instead of trying to compute in its head (where it might hallucinate). This shows how the **Action Interface** extends the LLM's capabilities beyond text generation.

---
# 6. Goal & Task Management, Reasoning & Planning

## Goal Management

In classical AI (Russell & Norvig's BDI architecture), goal management involves:
- **Beliefs** = what the agent knows (state of the world)
- **Desires** = what the agent wants (possible goals)
- **Intentions** = what the agent is committed to pursue (active goals)

### Why Is Goal Management Hard?

- **Conflicting goals**: "Be on time" vs. "Drive slowly for safety"
- **Dynamic environments**: A goal set earlier might no longer make sense
- **Scalability**: Adding new goals often required rewriting arbitration logic
- **No meta-goals**: Classical agents couldn't reflect and reprioritize

### Goal Management with LLMs

LLM-based systems handle goals differently:
- **Natural language goals** — users express them freely ("Plan my trip under €500")
- **Goal decomposition** — LLMs break high-level goals into subgoals
- **Dynamic reprioritization** — frameworks like AutoGPT re-evaluate at each step
- **Arbitration strategies**: rule-based filters, scoring functions, human-in-the-loop

## Reasoning Frameworks

LLMs are "local optimizers" — they generate the next most likely token. Multi-step tasks require explicit planning scaffolds:

| Framework | Description |
|---|---|
| **CoT** (Chain-of-Thought) | "Think step by step" — linear reasoning chain |
| **ToT** (Tree-of-Thought) | Explore multiple reasoning paths, evaluate and prune |
| **SoT** (Skeleton-of-Thought) | Generate outline first, then fill in details |
| **Self-Consistency** | Run same question multiple times, compare outputs |
| **ReAct** | Alternate between Reasoning and Acting (tool calls) |

## Control Loop

The agentic control loop follows a cybernetic cycle: **Plan → Act → Observe → Reflect → Repeat**

Let's demonstrate this with a **Planner + Worker** architecture:

In [ ]:
# Goal & Task Management: Planner + Worker architecture
planner = Agent(
    name="Planner",
    model="gpt-4o-mini",
    instructions="""You are a planning agent.
Given a high-level goal, break it into 3–7 ordered, concrete sub-tasks.
Output numbered steps.""",
)

worker = Agent(
    name="Worker",
    model="gpt-4o-mini",
    instructions="""You execute plans step-by-step.
You may call tools to do calculations or retrieve memory.
After each step, briefly say what you did.""",
    tools=[safe_calculator, retrieve_memory],
)

goal = """Create 3 bullet points that explain to MSc AI students
how LLMs and agentic systems complement each other. 
Then compute the average of exam scores [7.5, 8.0, 9.0]."""

# Step 1: Plan
print(">>> Planning phase")
plan_result = await Runner.run(planner, goal)
plan = plan_result.final_output
print(plan)

print("\n>>> Execution phase")
# Step 2: Execute the plan
exec_result = await Runner.run(worker, f"Execute this plan:\n{plan}")
print(exec_result.final_output)

**Observation:** This demonstrates the **control loop** — the Planner decomposes the high-level goal into concrete steps, and the Worker executes them sequentially, using tools when needed. This separation of planning and execution is a key pattern in agentic architectures.

Note how this mirrors classical AI planning (STRIPS, PDDL) but uses natural language for everything — goals, plans, and actions. The LLM's general knowledge eliminates the need for handcrafted domain models.

---
# 7. Verification & Guardrails

In multi-step agent loops, errors can **propagate and compound**. This is especially dangerous in high-stakes contexts (law, medicine, finance). We need:

| Component | Purpose |
|---|---|
| **Fact-checking modules** | Retrieval-Augmented Verification (RAV): decompose answers into atomic claims, verify each against retrieved evidence |
| **Safety filters** | Ethical guardrails (hate speech), legal guardrails (GDPR, copyright), domain-specific constraints |
| **External critics** | Self-consistency checks, debate agents, Reflexion frameworks (self-review of past mistakes) |

Let's create a **Critic Agent** that reviews another agent's output:

In [ ]:
# Verification: Critic agent reviews the Worker's output
critic = Agent(
    name="Critic",
    model="gpt-4o-mini",
    instructions="""You are a verification and safety critic.
Given an answer produced by another agent, you:
1. Check factual and mathematical consistency.
2. Point out likely hallucinations or unjustified claims.
3. Enforce classroom-appropriate tone (no hate, self-harm, etc.).

Respond in JSON with keys:
- "overall_ok": boolean
- "issues": list of strings (empty if no issues)
- "suggested_fix": revised answer if needed, or null if ok""",
)

answer_to_check = exec_result.final_output  # from the worker above
critic_result = await Runner.run(
    critic,
    f"Review this answer for factual accuracy and safety:\n\n{answer_to_check}",
)
print("Critic's assessment:")
print(critic_result.final_output)

**Observation:** The Critic agent acts as an independent verifier. It checks whether the Worker's output is factually correct, mathematically consistent, and safe. In production systems, this pattern prevents hallucinations from reaching the user.

This illustrates the concept of **agents checking other agents' work** — a key design pattern for reliable multi-step pipelines.

---
# 8. Multi-Agent Orchestration

## When and Why?

Multi-agent systems become valuable when tasks require more structure, reliability, or specialised reasoning than a single model can provide:

- **Specialisation** — different agents focus on different competencies (planner, researcher, writer, verifier)
- **Redundancy** — multiple agents critique each other's work, reducing hallucinations
- **Inspectable structure** — explicit, auditable reasoning steps

**However**, multi-agent systems come at a cost: more computation, more complexity, harder debugging. Only use them when the added orchestration genuinely improves performance.

## Orchestration Patterns

| Pattern | Description |
|---|---|
| **Hierarchical (Manager + Specialists)** | A central manager routes queries to specialist agents |
| **Pipeline** | Task flows through agents in sequence (planner → researcher → writer → editor) |
| **Debate / Committee** | Multiple agents answer independently; a judge reconciles |
| **Shared Blackboard** | Agents contribute to a common memory store others can read |

Let's demonstrate three patterns:

## 8.1 Manager + Specialists (Hierarchical Pattern)

We create:
- A `MathAgent` with calculator tools
- A `SearchAgent` that focuses on Information Retrieval explanations
- A `ManagerAgent` that routes questions to the right specialist via handoffs

In [ ]:
# Multi-Agent I: Manager + Specialists

# Specialist 1: Math
math_agent = Agent(
    name="MathAgent",
    model="gpt-4o-mini",
    instructions="""You are a precise math tutor. Show your reasoning,
but if you are unsure, say so and be conservative.""",
    tools=[safe_calculator],
)

# Specialist 2: IR/NLP
ir_agent = Agent(
    name="IRSearchAgent",
    model="gpt-4o-mini",
    instructions="""You are an expert in Information Retrieval and Text Mining.
You explain concepts such as embeddings, retrieval models (BM25, dense retrieval),
text classification, topic modeling, and conversational search.
Keep answers concise and suitable for MSc AI students.""",
)

# Manager: routes to specialists
@function_tool
def ask_math_agent(question: str) -> str:
    """Forward a math question to the Math specialist agent."""
    import asyncio
    result = asyncio.get_event_loop().run_until_complete(
        Runner.run(math_agent, question)
    )
    return result.final_output

@function_tool
def ask_ir_agent(question: str) -> str:
    """Forward an IR/NLP question to the IR specialist agent."""
    import asyncio
    result = asyncio.get_event_loop().run_until_complete(
        Runner.run(ir_agent, question)
    )
    return result.final_output

manager = Agent(
    name="ManagerAgent",
    model="gpt-4o-mini",
    instructions="""You are a routing manager. Given a user question, decide:
- If it's about math/calculations → use ask_math_agent
- If it's about IR, NLP, text mining → use ask_ir_agent
- If mixed → use both and combine the answers
Explain which agent you chose and why.""",
    tools=[ask_math_agent, ask_ir_agent],
)

# Test with different types of questions
questions = [
    "Compute the standard deviation of [7.5, 8.0, 9.0].",
    "Explain how embeddings help retrieve similar legal cases.",
    "For exam grading, I have scores [6, 7, 9]. What is the average, and how could an IR system help me analyse student answers?",
]

for q in questions:
    print(f"=== USER: {q} ===")
    r = await Runner.run(manager, q)
    print(r.final_output)
    print()

## 8.2 Debate / Committee Pattern

Two independent agents answer the same question. A `JudgeAgent` then critiques both answers and produces a final, hopefully better, response. This illustrates **external critics/evaluators**, **ensemble methods**, and **self-consistency**:

In [ ]:
# Multi-Agent II: Debate / Committee

answerer_a = Agent(
    name="AnswererA",
    model="gpt-4o-mini",
    instructions="""You are an optimistic, creative assistant.
Explain things intuitively, even if you need to guess a bit.""",
)

answerer_b = Agent(
    name="AnswererB",
    model="gpt-4o-mini",
    instructions="""You are a cautious, conservative assistant.
Avoid speculation; admit when something is unclear.""",
)

judge = Agent(
    name="JudgeAgent",
    model="gpt-4o-mini",
    instructions="""You are a critical judge.
Given a user question and two candidate answers:
1. Identify strengths and weaknesses of each.
2. Flag any likely hallucinations or mistakes.
3. Produce a final, improved answer combining the best of both.""",
)

question = "How can multi-agent LLM systems reduce hallucinations in legal document summarisation tasks?"

# Get two independent answers
ans_a = await Runner.run(answerer_a, question)
ans_b = await Runner.run(answerer_b, question)

# Judge evaluates both
judge_prompt = f"""User question: {question}

Answer A (optimistic):
{ans_a.final_output}

Answer B (cautious):
{ans_b.final_output}

Please evaluate both and produce a final consolidated answer."""

verdict = await Runner.run(judge, judge_prompt)
print("=== JUDGE'S FINAL ANSWER ===")
print(verdict.final_output)

## 8.3 Shared Memory / Blackboard Pattern

All agents can write notes to a shared long-term memory (FAISS index) and read from it. This mimics a classical **blackboard architecture** where agents coordinate through a common workspace:

In [ ]:
# Multi-Agent III: Shared Memory / Blackboard

@function_tool
def write_to_blackboard(note: str) -> str:
    """Any agent can post a note to the shared vector memory."""
    add_memory(note)
    log_event(f"Blackboard note: {note}")
    return "Note written to shared blackboard."

@function_tool
def read_from_blackboard(query: str) -> str:
    """Any agent can retrieve relevant notes from the shared vector memory."""
    res = search_memory(query, k=5)
    if not res:
        return "No relevant notes on the blackboard."
    return "\n".join(f"- {text} (score={score:.3f})" for text, score in res)

researcher = Agent(
    name="Researcher",
    model="gpt-4o-mini",
    instructions="""You are a research agent. Gather key points about a topic
and write them to the shared blackboard using write_to_blackboard.""",
    tools=[write_to_blackboard],
)

writer = Agent(
    name="Writer",
    model="gpt-4o-mini",
    instructions="""You are a writing agent. Read relevant notes from the 
shared blackboard using read_from_blackboard, then produce a well-structured
summary paragraph.""",
    tools=[read_from_blackboard],
)

# Phase 1: Researcher gathers and writes notes
print(">>> Research phase")
r_res = await Runner.run(
    researcher,
    "Gather key points about how LLMs and agentic systems complement each other in the context of conversational search. Write at least 3 notes to the blackboard.",
)
print(r_res.final_output)

# Phase 2: Writer reads blackboard and synthesizes
print("\n>>> Writing phase")
r_write = await Runner.run(
    writer,
    "Read the blackboard for notes about LLMs and agentic systems. Produce a concise summary suitable for MSc AI students.",
)
print(r_write.final_output)

**Observation:** In the Blackboard pattern, agents coordinate through a shared memory store rather than directly communicating. The Researcher writes facts; the Writer reads them and synthesizes. This pattern is particularly useful for complex pipelines where intermediate results need to be shared across multiple agents.

---
# 9. Agent Interoperability & Standards

As agentic systems proliferate, **interoperability** becomes crucial. Several standards and protocols are emerging:

## Agent Orchestration Frameworks

| Framework | Developer | Key Features |
|---|---|---|
| **OpenAI Agents SDK** | OpenAI | Tool calling, sessions, handoffs (used in this tutorial) |
| **LangGraph** | LangChain | Graph-based agent workflows, state machines |
| **Google ADK** | Google | Agent Development Kit with sub-agent routing |
| **Mistral Agents API** | Mistral | Agent orchestration for Mistral models |
| **N8n** | Open source | Visual multi-agent design and orchestration |
| **Amazon Bedrock** | AWS | Managed agent orchestration on AWS |

## Agent Communication Protocols

| Protocol | Purpose |
|---|---|
| **MCP** (Model Context Protocol) | Defines how LLMs interface with **tools** — search internet, access databases, orchestrate workflows. Standardizes the tool layer. |
| **A2A** (Agent-to-Agent) | Defines how agents **discover and communicate** with each other. Includes *Agent Cards* (JSON metadata: name, capabilities, skills). |
| **ANP** (Agent Network Protocol) | Handles network identities for agents |
| **ACP** (Agent Communication Protocol) | Standardizes messaging format between agents |

### Example: Google ADK Multi-Agent Setup

```python
from google.adk.agents import LlmAgent

billing_agent = LlmAgent(name="Billing", description="Handles billing inquiries.")
support_agent = LlmAgent(name="Support", description="Handles technical support.")

coordinator = LlmAgent(
    name="HelpDeskCoordinator",
    model="gemini-2.0-flash",
    instruction="Route user requests to the appropriate specialist.",
    sub_agents=[billing_agent, support_agent],
)
```

### Example: A2A Agent Card

```json
{
  "name": "LegalResearchAgent",
  "description": "Searches legal databases and summarizes case law.",
  "url": "https://legal-agent.example.com",
  "version": "1.0.0",
  "capabilities": { "streaming": true, "pushNotifications": false },
  "skills": [
    { "name": "case_search", "description": "Search for relevant legal cases" },
    { "name": "statute_lookup", "description": "Look up statutes by jurisdiction" }
  ]
}
```

## Agent Evaluation & Benchmarks

How do we know if agents actually work? Several benchmarks have emerged:

| Benchmark | What It Tests | Current Best |
|---|---|---|
| **WebArena** | Browser-based tasks (navigate, fill forms, search) | ~68% success |
| **TheAgentCompany** | Virtual company with 175 employee tasks | ~43% solved |
| **MultiAgentBench** | Multi-agent coordination tasks | Active research |

These numbers show that **agent systems are still far from reliable** in complex real-world tasks. The gap between demo performance and production reliability remains significant.

---
# 10. Challenges & Future Directions

## Current Challenges

| Challenge | Description |
|---|---|
| **Accuracy & Hallucinations** | Human-in-the-loop needed for high-stakes decisions |
| **Tool Misuse & Safety** | Sandboxing, whitelisting, audit trails required |
| **Bias & Fairness** | RLHF alignment doesn't guarantee fairness; testing for discriminatory outputs |
| **Privacy** | Agents handle sensitive data — need strict data governance |
| **Transparency** | Reveal AI identity, log reasoning/sources for accountability |
| **Human Oversight** | Augment, not replace — keep humans in the loop |
| **Evaluation** | Multi-step action sequences are hard to evaluate; new benchmarks needed |

## "There Is Nothing Magic"

LLM-based agents fit naturally into the **Reinforcement Learning** framework:
- **States** = conversation history + memory + environment
- **Actions** = generate text, call tools, ask clarifying questions
- **Policy** = the LLM itself (mapping states to actions)
- **Reward** = task completion, user satisfaction, factual accuracy

The key difference from classical RL: the action space is **combinatorial and expressed in natural language**.

## Future Directions

1. **Multimodal Conversational AI** — voice, text, and image search seamlessly combined
2. **Augmented Reality Search** — smart glasses overlaying information on the real world
3. **LLM + Structured Search Integration** — combining neural reasoning with database queries
4. **Ethical AI** — addressing energy consumption, misinformation, bias, privacy, and filter bubbles

---
# Summary: The LLM–Agent Symbiosis

**LLMs alone give us:**
- Perception over diverse text (and, with multimodal models, images/audio/video)
- Short-term conversation memory via the context window
- Powerful but myopic reasoning inside one prompt

**Agentic architectures add:**
- Tools and environment control → real actions in the world
- Structured memory (vector DBs, logs) beyond the context window
- Goal & task management, planning, critics, and guardrails

**Conversely, agentic scaffolding fixes several limitations of stand-alone LLMs:**
- Reduced hallucinations via tools, retrieval, and critics
- Longer-term coherence via explicit memories
- Better safety via explicit guardrail components

This is the **LLM–Agent symbiosis** that makes modern conversational search possible.

### References

| Paper | Authors |
|---|---|
| *The Rise and Potential of Large Language Model Based Agents* | Xi et al. (2023) |
| *ReAct: Synergizing Reasoning and Acting in Language Models* | Yao et al. (2022) |
| *TrustAgent: Towards Safe and Trustworthy LLM-based Agents* | Hua et al. (2024) |
| *Chain-of-Agents: LLM Collaboration for Long-Context Tasks* | Google Research (2025) |
| *PaSa: An LLM Agent for Comprehensive Academic Paper Search* | He et al. (2025, ACL) |

---
# Exercises

The following exercises are graded. Please provide your answers in the designated cells below.

## Exercise 1 — RAG vs Agentic Search (5 points)

Compare and contrast **standard RAG** (Retrieval-Augmented Generation) and **Agentic Search** as approaches to conversational information retrieval. In your answer, address:

1. What are the key architectural differences between standard RAG and an agentic search system?
2. What specific limitations of RAG does an agentic architecture overcome?
3. Give a concrete example of a search task that *requires* an agentic approach and explain why standard RAG would fail.

Write your answer in the cell below (minimum 150 words).

YOUR ANSWER HERE

YOUR ANSWER HERE

## Exercise 2 — Multi-Agent Orchestration Patterns (5 points)

This tutorial demonstrated three multi-agent orchestration patterns: **Manager + Specialists**, **Debate / Committee**, and **Shared Blackboard**. In your answer, address:

1. For each pattern, describe in one sentence what makes it distinct from the other two.
2. What are the trade-offs of multi-agent systems vs. a single well-prompted agent? Consider cost, latency, reliability, and complexity.
3. A law firm wants to build an AI system that (a) searches case law databases, (b) verifies legal citations, and (c) produces a summary memo for lawyers. Which orchestration pattern(s) would you recommend and why?

Write your answer in the cell below (minimum 150 words).

YOUR ANSWER HERE

YOUR ANSWER HERE

## Exercise 3 — Build a Fact-Checking Agent Pipeline (10 points)

Write code that implements a **two-agent fact-checking pipeline** using the OpenAI Agents SDK:

1. Create an `AnswerAgent` that answers a user question using `gpt-4o-mini`
2. Create a `FactCheckAgent` that:
   - Receives the `AnswerAgent`'s output
   - Decomposes it into individual factual claims
   - Evaluates each claim as "supported", "unsupported", or "unverifiable"
   - Returns a JSON string with keys `"claims"` (list of dicts with `"claim"` and `"verdict"`) and `"overall_trustworthy"` (boolean)
3. Run the pipeline on the question: `"What are the main differences between BM25 and dense retrieval for document search?"`
4. Store the AnswerAgent's output in a variable called `answer_text` and the FactCheckAgent's output in a variable called `fact_check_result`

You may use the `Agent`, `Runner`, and `await` pattern demonstrated in this tutorial.

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError("Replace this line with your solution")

In [ ]:
# Autograder test cell — do not modify
assert 'answer_text' in dir(), "You need to define 'answer_text'"
assert 'fact_check_result' in dir(), "You need to define 'fact_check_result'"
assert isinstance(answer_text, str) and len(answer_text) > 50, \
    "answer_text should be a non-trivial string response"
assert isinstance(fact_check_result, str) and len(fact_check_result) > 20, \
    "fact_check_result should be a non-trivial string response"
# Try parsing the fact-check result as JSON
import json
try:
    parsed = json.loads(fact_check_result)
    assert "claims" in parsed, "fact_check_result JSON should contain 'claims' key"
    assert "overall_trustworthy" in parsed, "fact_check_result JSON should contain 'overall_trustworthy' key"
    assert isinstance(parsed["claims"], list), "'claims' should be a list"
    assert len(parsed["claims"]) > 0, "'claims' list should not be empty"
    print(f"Answer length: {len(answer_text)} chars")
    print(f"Number of claims checked: {len(parsed['claims'])}")
    print(f"Overall trustworthy: {parsed['overall_trustworthy']}")
    print("All auto-graded tests passed!")
except json.JSONDecodeError:
    print("Warning: fact_check_result is not valid JSON, but we will accept non-JSON critic output too.")
    print(f"Answer length: {len(answer_text)} chars")
    print(f"Fact-check length: {len(fact_check_result)} chars")
    print("Auto-graded tests passed (non-JSON output accepted).")